In [3]:
import cv2
import os
import zipfile
import shutil
from tqdm import tqdm
from pathlib import Path

# Настройки
SOURCE_ZIP = '/content/drive/MyDrive/Датасет для обучения.zip'
TEMP_VIDEO_DIR = '/content/temp_videos'
OUT_FOLDER = '/content/student_dataset'
FINAL_ZIP = '/content/students_prepared.zip'
FRAME_INTERVAL = 10  # Каждый 10-й кадр
EXTENSIONS = ('.mp4', '.avi', '.mov', '.mkv')

def prepare_dataset():
    # Очистка старых папок
    for folder in [TEMP_VIDEO_DIR, OUT_FOLDER]:
        if os.path.exists(folder):
            shutil.rmtree(folder)
        os.makedirs(folder)

    # 1. Распаковка
    print(f"📦 Распаковка {SOURCE_ZIP}...")
    with zipfile.ZipFile(SOURCE_ZIP, 'r') as z:
        z.extractall(TEMP_VIDEO_DIR)

    # 2. Поиск ВСЕХ видео файлов (рекурсивно)
    video_paths = []
    for root, dirs, files in os.walk(TEMP_VIDEO_DIR):
        for file in files:
            if file.lower().endswith(EXTENSIONS):
                video_paths.append(os.path.join(root, file))

    if not video_paths:
        print("❌ ОШИБКА: Видео не найдены! Проверь содержимое архива.")
        return

    print(f"📸 Найдено видео: {len(video_paths)}. Начинаю нарезку...")

    for video_path in tqdm(video_paths):
        # Имя человека = имя файла без расширения
        person_name = Path(video_path).stem
        person_dir = os.path.join(OUT_FOLDER, person_name)
        os.makedirs(person_dir, exist_ok=True)

        cap = cv2.VideoCapture(video_path)
        count = 0
        saved_count = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret: break

            if count % FRAME_INTERVAL == 0:
                img_name = f"{person_name}_{saved_count:04d}.jpg"
                cv2.imwrite(os.path.join(person_dir, img_name), frame)
                saved_count += 1
            count += 1

        cap.release()

    # 3. Создание финального ZIP для скачивания
    print("📦 Упаковка в ZIP...")
    with zipfile.ZipFile(FINAL_ZIP, 'w', zipfile.ZIP_DEFLATED) as z:
        for root, dirs, files in os.walk(OUT_FOLDER):
            for file in files:
                rel_path = os.path.relpath(os.path.join(root, file), OUT_FOLDER)
                z.write(os.path.join(root, file), rel_path)

    print(f"✅ Готово! Ссылка на файл: {FINAL_ZIP}")

prepare_dataset()

📦 Распаковка /content/drive/MyDrive/Датасет для обучения.zip...
📸 Найдено видео: 2. Начинаю нарезку...


100%|██████████| 2/2 [00:07<00:00,  3.67s/it]


📦 Упаковка в ZIP...
✅ Готово! Ссылка на файл: /content/students_prepared.zip
